### Coleta e persistência dos dados neste notebook

**tipo de ingestão**: full load (recomendado para ingestão de dados inicial no datalake)

**origem**: Yahoo finance, Wikipedia, Fear and Greed (alternative.me)

**destino**: bronze/database/bitcoin/raw/btc.csv

**formato do data no data lake**: .csv


#### Instruções

1. Renomear o arquivo .env_exemplo para somente .env
2. Adicionar popular as variaveis conforme o padrão de nomenclatura que voce utilizar

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from transformers import pipeline
import yfinance as yf
import mwclient
import time
import requests
import json
from datetime import datetime
from statistics import mean
from io import StringIO
from dotenv import load_dotenv
import os

##carrega variaveis de ambiente
load_dotenv()

c:\Users\julio\OneDrive\Área de Trabalho\Projects\bitcoin_trend_prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

Extrai os dados de bitcoin do Yahoo Finance

In [13]:
def extract_btc(data_inicio: datetime) -> pd.DataFrame:
    ticker = yf.Ticker("BTC-USD")
    btc = ticker.history(start=data_inicio)
    return btc

Formata a base de bitcoin

In [33]:
def format_base(df:pd.DataFrame) -> pd.DataFrame:
    df.index = pd.to_datetime(df.index).tz_localize(None)
    del df["Dividends"]
    del df["Stock Splits"]
    df.columns = [c.lower() for c in df.columns]
    return df

In [56]:
data_inicio = datetime.strptime('2018-02-01', '%Y-%m-%d')
btc = format_base(extract_btc(data_inicio))
btc.head()

,open,high,low,close,volume
Date,,,,,
2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9959400448
2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,12726899712
2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,7263790080
2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,7073549824
2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,9285289984


Extrai os dados das reviews de bitcoin da wikipedia

In [15]:
def extract_reviews() -> list:
   site = mwclient.Site('en.wikipedia.org')
   page = site.pages['Bitcoin']
   revs = list(page.revisions(start=data_inicio, dir='newer'))
   revs = sorted(revs, key=lambda rev: rev["timestamp"])
   return revs

Classifica o sentimento relacionado as reviews

seleciona o modelo utilizado para classificação das reviews

In [16]:
sentiment_pipeline = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\julio\OneDrive\Área de Trabalho\Projects\bitcoin_trend_prediction\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


transforma o score das classificações negativas em valores negativos

In [17]:
def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

cria a base de sentimentos

In [57]:
edits = {}
revs = extract_reviews()

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

prepara e limpa a base

In [58]:
for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [21]:
def create_edits_df(edits: dict) -> pd.DataFrame:
    edits_df = pd.DataFrame.from_dict(edits, orient="index")
    edits_df.index = pd.to_datetime(edits_df.index)
    dates = pd.date_range(start=data_inicio, end=datetime.today())
    edits_df = edits_df.reindex(dates, fill_value=0)
    rolling_edits = edits_df.rolling(30, min_periods=30).mean()
    rolling_edits = rolling_edits.dropna()
    return rolling_edits

In [59]:
rolling_edits = create_edits_df(edits)
rolling_edits.head()

,edit_count,sentiment,neg_sentiment
2018-03-02,3.066667,-0.307224,0.541296
2018-03-03,3.066667,-0.307224,0.541296
2018-03-04,2.933333,-0.258349,0.513519
2018-03-05,2.866667,-0.259235,0.496852
2018-03-06,2.666667,-0.249632,0.474630


Extrai a base de Fear and Greed da Alternative.me

In [24]:
def collect_fear_greed(api_url:str, limit:int, drop_colls:list, rename_colls:list) -> pd.DataFrame:
    response = requests.get(f'{api_url}?limit={limit}')
    n_data = response.json()
    df = pd.read_json(json.dumps(n_data['data']))
    df.drop(columns=drop_colls, axis=1, inplace=True)
    df.rename(columns=rename_colls, inplace=True)
    df.set_index('date', inplace=True)
    return df

In [60]:
api_url = 'https://api.alternative.me/fng/'
colls_to_drop = ['time_until_update']
coll_names = {'value':'fng_index', 
              'value_classification':'fng_class',
              'timestamp': 'date'}

data_fg = collect_fear_greed(api_url=api_url, limit=0, drop_colls=colls_to_drop, rename_colls=coll_names)

C:\Users\julio\AppData\Local\Temp\ipykernel_16468\2951827581.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(n_data['data']))


In [61]:
data_fg = data_fg.sort_index()

Une as bases de bitcoin, sentimentos e fear and greed

In [62]:
btc = btc.merge(rolling_edits, left_index=True, right_index=True)
btc = btc.merge(data_fg, left_index=True, right_index=True)
btc.head()

,open,high,low,close,volume,edit_count,sentiment,neg_sentiment,fng_index,fng_class
2018-03-02,10977.400391,11189.000000,10850.099609,11086.400391,7620590080,3.066667,-0.307224,0.541296,47,Neutral
2018-03-03,11101.900391,11528.200195,11002.400391,11489.700195,6690570240,3.066667,-0.307224,0.541296,56,Greed
2018-03-04,11497.400391,11512.599609,11136.099609,11512.599609,6084149760,2.933333,-0.258349,0.513519,44,Fear
2018-03-05,11532.400391,11704.099609,11443.900391,11573.299805,6468539904,2.866667,-0.259235,0.496852,55,Greed
2018-03-06,11500.099609,11500.099609,10694.299805,10779.900391,6832169984,2.666667,-0.249632,0.474630,59,Greed


SUGESTÃO: se for trabalhar nos dados a partir deste ponto, descomente a celula abaixo para gerar um backup do dataframe para agilizar seu trabalho

In [77]:
# work_backup = btc.copy()
# work_backup.head()

Prepara Coluna Target e Colunas adicionais

In [75]:
def improve_df(df: pd.DataFrame) -> pd.DataFrame:
    df["tomorrow"] = df["close"].shift(-1)
    df["fng_in_yest"] = df["fng_index"].shift(1)
    df["fng_class_yest"] = df["fng_class"].shift(1)
    df["target"] = (df["tomorrow"] > df["close"]).astype(int)
    df.dropna(inplace=True)
    return df

Organiza as colunas

In [78]:
# btc = work_backup.copy()
btc = improve_df(btc)
sequencia = ['open', 'high', 'low', 'close', 'volume', 'edit_count', 'tomorrow','sentiment',
       'neg_sentiment', 'fng_index', 'fng_class','fng_in_yest', 'fng_class_yest', 'target']
btc = btc[sequencia]
btc.head()

,open,high,low,close,volume,edit_count,tomorrow,sentiment,neg_sentiment,fng_index,fng_class,fng_in_yest,fng_class_yest,target
2018-03-04,11497.400391,11512.599609,11136.099609,11512.599609,6084149760,2.933333,11573.299805,-0.258349,0.513519,44,Fear,56.0,Greed,1
2018-03-05,11532.400391,11704.099609,11443.900391,11573.299805,6468539904,2.866667,10779.900391,-0.259235,0.496852,55,Greed,44.0,Fear,0
2018-03-06,11500.099609,11500.099609,10694.299805,10779.900391,6832169984,2.666667,9965.570312,-0.249632,0.474630,59,Greed,55.0,Greed,0
2018-03-07,10803.900391,10929.500000,9692.120117,9965.570312,8797910016,2.633333,9395.009766,-0.216476,0.441296,37,Fear,59.0,Greed,0
2018-03-08,9951.440430,10147.400391,9335.870117,9395.009766,7186089984,2.600000,9337.549805,-0.188242,0.407963,39,Fear,37.0,Fear,0


Realiza a ingestão do dataframe no data lake

In [79]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [80]:
# Converte o dataframe em uma string CSV
csv_buffer = StringIO()
btc.to_csv(csv_buffer, index=True)

In [81]:
bucket_name = os.environ['BUCKET_NAME']
csv_file_name = os.environ['CSV_FILE_NAME']
bucket_layer = os.environ['BUCKET_LAYER']
object_name = f'{bucket_layer}/{csv_file_name}'

In [82]:
def upload_to_s3(bucket_name, object_name, csv_buffer):
    try:
        s3_resource = boto3.resource('s3')
        req_metadata = s3_resource.Object(bucket_name, object_name).put(Body=csv_buffer.getvalue())
        status = req_metadata['ResponseMetadata']['HTTPStatusCode']
        if status == 200:
            print('Upload sucessful')
        else:
            print(f'Upload failed, status {status}')
    except ClientError as e:
        logging.error(e)

upload_to_s3(bucket_name, object_name, csv_buffer)

Upload sucessful


In [34]:
# baixa csv localmente
#btc.to_csv('btc.csv')